In [103]:
import numpy as np
import pandas as pd
import lightgbm as lgb
from sklearn.metrics import mean_absolute_error
from sklearn import preprocessing
pd.options.display.max_columns=999

In [104]:
train=pd.read_csv('train.tsv',delimiter='\t')
test=pd.read_csv('test.tsv',delimiter='\t')

In [105]:
df=train.append(test).reset_index(drop=True)

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\frame.py:6211: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


In [106]:
df.head()

,acceleration,car name,cylinders,displacement,horsepower,id,model year,mpg,origin,weight
0,16.0,dodge aries se,4,135.0,84.00,0,82,29.0,1,2525.0
1,14.0,vw rabbit custom,4,89.0,71.00,3,79,31.9,2,1925.0
2,15.5,toyota mark ii,6,156.0,108.0,9,76,19.0,3,2930.0
3,14.5,dodge colt,4,90.0,75.00,11,74,28.0,1,2125.0
4,17.3,toyota tercel,4,89.0,62.00,13,81,37.7,3,2050.0


In [107]:
def split_name(text,num):
    split=text.split()
    try:
        return split[num]
    except:
        return np.nan

In [108]:
df['car0']=df['car name'].apply(lambda x:split_name(text=x,num=0))

In [109]:
df['car1']=df['car name'].apply(lambda x:split_name(text=x,num=1))
df['car2']=df['car name'].apply(lambda x:split_name(text=x,num=2))

In [110]:
car1_notnull=df['car1'].notnull()
car2_notnull=df['car2'].notnull()

In [111]:
df['car0_1']=df['car0'][car1_notnull]+df['car1'][car1_notnull]

In [112]:
df['car0_1_2']=df['car0'][car2_notnull]+df['car1'][car2_notnull]+df['car2'][car2_notnull]

In [113]:
df.drop(columns='car name',inplace=True)

In [114]:
df['horsepower']=df['horsepower'].replace('?',np.nan)
df['horsepower']=df['horsepower'].astype(np.float64)

In [115]:
temp=df[['mpg','car0']]
df['car0']=temp.groupby('car0').transform(np.mean).values

In [116]:
temp=df[['mpg','car0_1']]
df['car0_1']=temp.groupby('car0_1').transform(np.mean).values

In [117]:
temp=df[['mpg','car0_1_2']]
df['car0_1_2']=temp.groupby('car0_1_2').transform(np.mean).values

In [118]:
df.drop(columns=['car1','car2'],inplace=True)

In [119]:
df.head()

,acceleration,cylinders,displacement,horsepower,id,model year,mpg,origin,weight,car0,car0_1,car0_1_2
0,16.0,4,135.0,84.0,0,82,29.0,1,2525.0,22.144444,27.400000,29.0
1,14.0,4,89.0,71.0,3,79,31.9,2,1925.0,36.450000,34.133333,31.9
2,15.5,6,156.0,108.0,9,76,19.0,3,2930.0,30.130769,19.500000,19.5
3,14.5,4,90.0,75.0,11,74,28.0,1,2125.0,22.144444,27.000000,NaN
4,17.3,4,89.0,62.0,13,81,37.7,3,2050.0,30.130769,37.700000,NaN


In [120]:
df.to_csv('df_car_target_0.csv',index=None)

In [121]:
df.drop(columns=['car0_1_2'],inplace=True)

In [122]:
df.to_csv('df_car_target_1.csv',index=None)

In [123]:
df.drop(columns=['car0_1'],inplace=True)

In [124]:
df.to_csv('df_car_target_2.csv',index=None)

In [125]:
#平均とるとき自分除きたい